# Flipping coin

In [3]:
from ipywidgets import interact
import numpy as np
from scipy import stats
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

# This outputs the interactive plot in the notebook rather than creating a new html file
output_notebook()

# For plotting functions' curves
x = np.linspace(0, 1, 1000)

# Initiate it with uninformed distribution--uniform
y = stats.beta.pdf(x,1,1)

# Create a plot
p = figure(title="Posterior is proportinal to Likelihood times Prior", plot_height=400,
           plot_width=800, y_range=(-1,5))

# Likelihood, determined by the number of heads and tails we had through observation
likeli = p.line(x, y, color="black", legend = 'Likelihood', line_width=3)

# Current ratio or frequency of obtaining "head"
cur = p.line(x, y, color="black", line_width=3)

# Posterior distribution
post = p.line(x, y, color="red", legend = 'Posterior',line_width=3)

# Prior distribution, determined by the input alpha and beta
prior = p.line(x, y, color="green", legend = 'Prior', line_width=3)

show(p, notebook_handle=True)

# This function will take user's input and reflect it on the current plot--real-time actions
# When you edit the inputs, please select them first instead of deleting them
def update_coin(head, tail, alpha=1, beta=1):
    # Conver the input from string to integers
    head = int(head)
    tail = int(tail)
    
    # Update the three functions' curves
    likeli.data_source.data['y'] = stats.beta.pdf(x, head, tail)
    prior.data_source.data['y'] = stats.beta.pdf(x, alpha, beta)
    post.data_source.data['y'] = stats.beta.pdf(x, head+alpha, tail+beta)
    
    # Highlight the current ratio of obtaining "head" and update the "cur" accordingly
    p = 1.0*head/(head+tail)
    cur.data_source.data['x'] = [p,p]
    cur.data_source.data['y'] = [0,stats.beta.pdf(p, head, tail)]
    push_notebook()

Loading BokehJS ...

TypeError: show() got an unexpected keyword argument 'notebook_handle'

In [4]:
# "Head" and "Tail" should be non-negative, while "alpha" and "beta" here are range
# Feel free to change the upper bound of "alpha" and "beta" (default 10)

interact(update_coin, head = '6', tail = '4', alpha=(0.1,10), beta=(0.1,10))

<function __main__.update_coin>

# Normal distribution

$X \sim N(\hat\mu, \hat\sigma^2)$ sample data

$\hat\mu$ is the sample mu, which is unknow and $\hat\mu \sim N(\mu,\sigma)$--prior  

Sample sigma $\hat\sigma$ is known, we set it as follows:

In [3]:
sample_sigma = 2

In [4]:
output_notebook()

# Similar to previous one, except this time the y is standard normal as default setting
x = np.linspace(-10, 10, 1000)
y = stats.norm.pdf(x, 0, 1)

p = figure(title="Posterior is proportinal to Likelihood times Prior", plot_height=400,
           plot_width=800, y_range=(-0.1,0.6))

likeli = p.line(x, y, color="black", legend = 'Likelihood', line_width=3)
# "Cur" here indicates the current observation data point we are adding
cur = p.line(x, y, color="black", line_width=3)
post = p.line(x, y, color="red", legend = 'Posterior',line_width=3)

# The following is for a step-by-step comparison:
# We consider the posterior with n-1 points as the "prior" for n case, observe the difference
post_prior = p.line(x, y, color="red", line_width=3, line_dash = 'dashed')
prior = p.line(x, y, color="green", legend = 'Prior', line_width=3)
show(p, notebook_handle=True)

# Let python generate 5 random numbers, feel free to change it to whatever you like
data_num = 5

def update_normal(add_num = 1, mu=0, sigma=1):
    # These two parameters control the prior knowledge, default 0,1 (standar normal)
    mu = int(mu)
    sigma = int(sigma)
    # add_num controls which data point we are currently adding, for example:
    # if add_num = 3, then we have already added 2 points and we are about to include the 3rd
    
    # Control the random seed so adding observations won't change the dataset
    np.random.seed(2016+sigma+mu)
    
    # Sample mu is normally distributed with parameters mu and sigma (prior)
    sample_mu = np.random.normal(mu, sigma)
    
    # Generate sample data points with the sample_mu and sample_sigma
    dataset = np.random.normal(sample_mu, sample_sigma, size = data_num)
    
    # Update the prior
    prior.data_source.data['y'] = stats.norm.pdf(x, mu, sigma)
    
    # Update the likelihood
    likeli.data_source.data['y'] = stats.norm.pdf(x, sample_mu, sample_sigma)
    
    # Highlight the current data point by drawing a black line indicating its position on x axis
    cur.data_source.data['x'] = [dataset[add_num-1],dataset[add_num-1]]
    cur.data_source.data['y'] = [0, stats.norm.pdf(dataset[add_num-1], sample_mu, sample_sigma)]
    
    # Formulae can be found on the Internet, then update the posterior distribution
    post_sigma = 1.0/np.sqrt(sigma**(-2.) + 1.0*add_num / sample_sigma**2)
    post_mu = post_sigma**2 * (1.0*mu/sigma**2 + dataset[:add_num].sum() / sample_sigma**2)
    post.data_source.data['y'] = stats.norm.pdf(x, post_mu, post_sigma)
    
    # Update the "previous" posterior, which is determined by n-1 points  
    post_sigma_prior = 1.0/np.sqrt(sigma**(-2.) + 1.0*(add_num-1) / sample_sigma**2)
    post_mu_prior = post_sigma_prior**2 * (1.0*mu/sigma**2 + dataset[:add_num-1].sum() / sample_sigma**2)
    post_prior.data_source.data['y'] = stats.norm.pdf(x, post_mu_prior, post_sigma_prior)
    
    # Showing all random data points we've generated
    print(dataset)
    print('\nSample mu: %f'%sample_mu)
    print('Sample mean: %f'%dataset.mean())
    print('Posterior mu: %f'%post_mu)
    print('Posterior sigma: %f'%post_sigma)
    push_notebook()

Loading BokehJS ...

In [5]:
interact(update_normal, add_num = (1, data_num), mu='0', sigma='1')

[-1.30374127 -0.62476074  0.12400667 -1.812101   -2.07943002]

Sample mu: -1.022945
Sample mean: -1.139205
Posterior mu: -0.260748
Posterior sigma: 0.894427


<function __main__.update_normal>